### Intial Testing of the Dataset

In [10]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer

In [ ]:
df = pd.read_csv('imdb_top_1000.csv')
df.at[966,"Released_Year"] = 1995 #fixing the wrong value for apollo 13 
df.drop(columns = ["Poster_Link"], inplace= True)
df.dropna(inplace = True) #dropping null valued columns


FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb_top_1000.csv'

In [12]:
#Converting to Numeric Values
df["Gross"] = df["Gross"].str.replace(",", "").astype("float")
df["Released_Year"] = df["Released_Year"].astype("int")
df["Runtime"] = df["Runtime"].str.replace(" min", "").astype("int")
df["IMDB_Rating"] = df["IMDB_Rating"].astype(float)
df["Meta_score"] = df["Meta_score"].astype(float)
df["No_of_Votes"] = df["No_of_Votes"].astype(float)
df.head()


AttributeError: Can only use .str accessor with string values!

In [9]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")




NameError: name 'smooth_target_encode' is not defined

### Trying One Hot Encoding
Thoughts - Too many features \
Results - Not Tried

In [218]:
#One Hot encoding for Certificate, Genre, Director, Star1, Star2, Star3, Star 4
df_encoded = pd.get_dummies(df, columns = ["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"])
print(f"df_encoded_shape{df_encoded.shape}")
print(f"df_shape{df.shape}")

df_encoded_shape(714, 2953)
df_shape(714, 15)


### Trying Target Encoding

In [219]:
#defining functino to encode the target variable
def smooth_target_encode(df, col, target, smoothing_param =.3):
    mean_target = df[target].mean()
    encoded =df.groupby(col)[target].agg(["count", "mean"])
    counts = encoded["count"]
    means = encoded["mean"]
    smooth_encodings = (means*counts + mean_target*smoothing_param)/(counts+smoothing_param)
    return df[col].map(smooth_encodings)

In [220]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")

In [221]:
df.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,3.991965e+07,1994,6.605757e+07,142,3.478698e+07,3.991965e+07,Two imprisoned men bond over a number of years...,1.049399e+08,8.204214e+07,3.991965e+07,3.991965e+07,3.991965e+07,3.991965e+07,3.991965e+07,28341469.0
1,1.219388e+08,1972,6.605757e+07,175,3.328097e+07,1.219388e+08,An organized crime dynasty's aging patriarch t...,2.578054e+07,6.988265e+07,7.309586e+07,5.893506e+07,1.219388e+08,9.383499e+07,1.219388e+08,134966411.0
2,4.295481e+08,2008,1.499447e+08,152,7.275734e+07,1.879008e+08,When the menace known as the Joker wreaks havo...,8.741143e+07,2.362660e+08,1.656802e+08,2.788940e+08,4.295481e+08,1.803062e+08,4.295481e+08,534858444.0
3,6.219544e+07,1974,6.605757e+07,202,3.328097e+07,1.879008e+08,The early life and career of Vito Corleone in ...,7.654053e+07,6.988265e+07,4.794513e+07,1.465885e+08,7.144591e+07,9.383499e+07,6.219544e+07,57300000.0
4,1.110124e+07,1957,9.459847e+07,96,3.328097e+07,1.879008e+08,A jury holdout attempts to prevent a miscarria...,6.266548e+07,3.760792e+07,1.008806e+07,1.110124e+07,1.403221e+07,1.110124e+07,1.110124e+07,4360000.0


### Using Bert embeddings for overview column, Unesecarry for RF or Xgbost

In [ ]:

# Load a lightweight BERT model (fast & efficient)
model = SentenceTransformer('all-MiniLM-L6-v2')  # Outputs 384D embeddings

# Convert each overview into a 384-dimensional vector
df["Overview"] = df['Overview'].apply(lambda x: model.encode(x))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [225]:
df.head()
df.shape

(714, 15)